## Plotting an animation of your results 

In [ ]:
import xarray as xr
from datetime import timedelta as delta
import numpy as np 
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.animation as animation

In [ ]:
# enter your result file location and name 
data_xarray = xr.open_dataset(r'filename.nc') 

# this should be the same as your model dt
outputdt = delta(hours=6) 

# creating timerange variable (in nanoseconds)
timerange = np.arange(np.nanmin(data_xarray['time'].values),
                      np.nanmax(data_xarray['time'].values)+np.timedelta64(outputdt), 
                      outputdt)  

fname = r'ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp'

# set up figure options 
fig = plt.figure(figsize=(5,5),constrained_layout=True)
proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)

# set up grid and axis tick labels
gridlines = ax.gridlines(draw_labels=True,alpha=0.2,edgecolor='lightgray')
gridlines.xlabels_top = None
gridlines.ylabels_right = None

plt.xlim([95, 140])
plt.ylim([-12, 10])

# plot particles
time_id = np.where(data_xarray['time'] == timerange[0]) # Indices of the data where time = 0
scatter = ax.scatter(data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id],s=1,facecolor='red')

# plot coastlines
adm1_shapes = list(shpreader.Reader(fname).geometries())
ax.add_geometries(adm1_shapes, ccrs.PlateCarree(), edgecolor='darkgrey', facecolor='lightgray')

# set title 
t = str(timerange[0])
title = ax.set_title('Particles at '+t)

# outline which sections of the plot to animate
def animate(i):
    t = str(timerange[i])
    title.set_text('Particles at '+t)
    
    time_id = np.where(data_xarray['time'] == timerange[i])
    scatter.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])
    
# write animation
anim = FuncAnimation(fig, animate, frames = len(timerange), interval=100)

# find the ffmpeg.exe file within your py3_parcels environment in your file finder and copy the address to here
plt.rcParams['animation.ffmpeg_path'] =r'/Users/niajones/opt/anaconda3/envs/py3_parcels/bin/ffmpeg'
writervideo = animation.FFMpegWriter() 

#identify the filename you want to use and where you want to save it 
anim.save(r'6monthtest.mp4', writer=writervideo,dpi=300)

# write the video in html so it appears in this notebook as well 
HTML(anim.to_jshtml())
